In [1]:
import sqlite3
import pandas as pd
import logging
from contextlib import contextmanager


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S', 
    handlers=[
        logging.FileHandler("score_rh.log"),  # Salva logs em um arquivo
        logging.StreamHandler()  # Exibe logs no console
    ]
)

DB_PATH = "/app/data/teste___desafio_técnico_-_analista_de_dados_pleno_anexo_db.db"
SQL_SCRIPT_PATH = "/app/scripts/canditados_pre_selecionados.sql"
OUTPUT_CSV = "/app/output/candidatos_pre_selecionados.csv"

# Proximos passos 

* Verificar saida dos resultados 
* Criar Logica de candidato que se inscreve em muitas vagas 
* Desenvolver processo de automacao 
* Melhorar eligibilidade do codigo 
* Melhorar Arquitetura de projeto

# Conexao DB

In [2]:
@contextmanager
def conectar_banco():
    """Gerenciador de contexto para conexão com o banco de dados."""
    conn = None
    try:
        conn = sqlite3.connect(DB_PATH)
        yield conn  # Retorna a conexão para ser usada no bloco `with`
    except Exception as e:
        logging.error(f"Erro ao conectar ao banco de dados: {e}")
        raise
    finally:
        if conn:
            conn.close()  # Fecha a conexão após o uso

Modificar tabela vaga competencia coluna tempo experiencia para somente meses
Modificar tabela competencia_experiencia coluna tempo_competencia de dias para meses

# DATA PREP

In [3]:
def listar_tabelas():
    """Lista todas as tabelas do banco SQLite."""
    tabelas = []
    try:
        with conectar_banco() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tabelas = cursor.fetchall()
        return [t[0] for t in tabelas]
    except Exception as e:
        logging.error(f"Erro ao listar tabelas: {e}")
        return []

def visualizar_tabela(nome_tabela, haslimit=True, limite=5):
    try:
        with conectar_banco() as conn:
            if haslimit:
                query = f"SELECT * FROM {nome_tabela} LIMIT {limite};"
            else:
                query = f"SELECT * FROM {nome_tabela};"
            df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        logging.error(f"Erro ao visualizar tabela {nome_tabela}: {e}")
        return pd.DataFrame()

def verificar_nulos():
    """Identifica colunas com valores nulos em todas as tabelas."""
    tabelas = listar_tabelas()
    for tabela in tabelas:
        df = visualizar_tabela(tabela, haslimit=False)
        
        null_counts = df.isnull().sum()
        # Filtra apenas as colunas com valores nulos
        colunas_com_nulos = null_counts[null_counts > 0]
        
        if not colunas_com_nulos.empty:
            logging.warning(f"Na tabela {tabela}, as seguintes colunas têm valores nulos:")
            for coluna, quantidade in colunas_com_nulos.items():
                logging.warning(f"  - {coluna}: {quantidade} valores nulos")
        else:
            logging.info(f"Na tabela {tabela}, não há valores nulos.")


In [4]:
verificar_nulos()

2025-02-13 18:54:40 - INFO - Na tabela competencias, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela vagas, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela candidatos, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela vagacompetencia, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela candidato_vaga, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela experiencias, não há valores nulos.
2025-02-13 18:54:40 - INFO - Na tabela competencia_experiencia, não há valores nulos.


In [5]:
def executar_sqlite_query():
    try:
        with conectar_banco() as conn:
            cursor = conn.cursor()
            with open(SQL_SCRIPT_PATH, "r", encoding="utf-8") as f:
                sql_query = f.read()

            cursor.execute("PRAGMA foreign_keys = ON;") 
            df = pd.read_sql_query(sql_query, conn) 
            df = df.sort_values("candidato_id")

        df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        logging.info(f"Arquivo salvo com sucesso em: {OUTPUT_CSV}")
        return df

    except Exception as e:
        logging.error(f"Erro ao executar a consulta: {e}")
        return pd.DataFrame()

df_min_candidatos = executar_sqlite_query()

2025-02-13 18:54:45 - INFO - Arquivo salvo com sucesso em: /app/output/candidatos_pre_selecionados.csv


In [6]:
df_min_candidatos.head(10)

,candidato_id,candidato_nome,vaga_id,vaga_titulo,atende_localizacao,atende_salario,atende_fit_cultural,soma_competencias_atendidas,status
378,1,Lorena Pereira,44,vaga 44,0,0,0,2,Aprovado
117,2,Dr. Lorenzo Rocha,14,vaga 14,0,0,0,0,Inconsistência
339,3,Brayan Pastor,40,vaga 40,0,0,0,1,Aprovado
176,3,Brayan Pastor,21,vaga 21,0,0,0,2,Aprovado
407,4,Lucas Gabriel Nunes,47,vaga 47,0,1,0,0,Aprovado
571,6,Juliana Nascimento,69,vaga 69,0,0,0,0,Inconsistência
438,8,Bianca Cavalcanti,49,vaga 49,0,0,0,1,Aprovado
599,12,Mariana Mendonça,72,vaga 72,0,0,0,0,Inconsistência
63,14,Raul Marques,10,vaga 10,0,1,0,1,Aprovado
209,15,Davi Miguel Almeida,25,vaga 25,0,0,0,1,Aprovado


In [ ]:
# preciso das tabelas vagas, candidato_vaga, vaga competencia

In [12]:
# Função principal para calcular os scores
def calcular_score_salario(situacao_candidato, vagas_info: pd.DataFrame, candidato_vaga_info: pd.DataFrame):
    vaga_id = situacao_candidato['vaga_id']
    candidato_id = situacao_candidato['candidato_id']

    # Filtrar a vaga correspondente
    vaga = vagas_info.loc[vagas_info['id'] == vaga_id]
    candidato = candidato_vaga_info.loc[candidato_vaga_info['id_candidato'] == candidato_id]

    # Garantir que encontramos os dados antes de prosseguir
    if vaga.empty or candidato.empty:
        return 0

    salario_min = vaga['salario_minimo'].values[0]
    salario_max = vaga['salario_maximo'].values[0]
    pretensao_salarial = candidato['pretensao_salarial'].values[0]

    # Cálculo do score baseado na distância do salário ideal
    distancia_min = abs(pretensao_salarial - salario_min)
    distancia_max = abs(salario_max - pretensao_salarial)
    score = 20 * (min(distancia_min, distancia_max) / ((salario_max - salario_min) / 2))

    return round(score, 2)  # Retorna o score arredondado


def calcular_score_competencia(situacao_candidato, vagacompetencias_info: pd.DataFrame):
    vaga_id = situacao_candidato['vaga_id']
    soma_competencia_candidato = situacao_candidato['soma_competencias_atendidas']

    # Filtrar apenas as competências da vaga correspondente
    competencias_vaga = vagacompetencias_info[vagacompetencias_info['id_vaga'] == vaga_id]

    if competencias_vaga.empty:
        return 0

    # Contar quantas competências são exigidas pela vaga
    total_competencias_vaga = competencias_vaga['id_competencia'].nunique()

    # Definir peso proporcional das competências atendidas pelo candidato
    peso_por_competencia = 30 / total_competencias_vaga  # 30 pontos divididos entre as competências exigidas
    score = soma_competencia_candidato * peso_por_competencia

    return round(score, 2)  # Retorna o score arredondado

def calcular_scores(df_min_candidatos: pd.DataFrame): 
    df_min_incosistencias = df_min_candidatos[df_min_candidatos['status'] == 'Inconsistência']
    df_min_incosistencias['score'] = 1
    df_min_aprovados = df_min_candidatos[df_min_candidatos['status'] == 'Aprovado'] 

    if df_min_aprovados.empty:
        print("Nenhum candidato aprovado encontrado. Encerrando a execução.")
        return

    # Pegando apenas valores únicos
    candidatos_ids = list(set(df_min_aprovados['candidato_id'].tolist()))
    vagas_ids = list(set(df_min_aprovados['vaga_id'].tolist()))  # Aqui estava errado, agora pega os IDs das vagas!

    # Formatando os IDs como string para uso na query
    candidatos_ids_str = ",".join(map(str, candidatos_ids))
    vagas_ids_str = ",".join(map(str, vagas_ids))

    with conectar_banco() as conn:
        # Filtrar apenas as vagas que possuem candidatos aprovados
        query_vagas = f"""
        SELECT * FROM vagas 
        WHERE id IN ({vagas_ids_str})
        """
        vagas_info = pd.read_sql_query(query_vagas, conn)

        # Filtrar apenas os registros de candidato_vaga relevantes
        query_candidato_vaga = f"""
        SELECT * FROM candidato_vaga 
        WHERE id_candidato IN ({candidatos_ids_str}) AND id_vaga IN ({vagas_ids_str})
        """
        candidato_vaga_info = pd.read_sql_query(query_candidato_vaga, conn)

        # Filtrar apenas as competências das vagas onde os candidatos foram aprovados
        query_vaga_competencias = f"""
        SELECT * FROM vagacompetencia 
        WHERE id_vaga IN ({vagas_ids_str})
        """
        vagacompetencias_info = pd.read_sql_query(query_vaga_competencias, conn)

    # Criar lista para armazenar os scores
    scores = []

    for _, row in df_min_aprovados.iterrows():  # Corrigido para iterar corretamente sobre as linhas do DataFrame
        score = 0

        if row['atende_localizacao'] == 1:
            score += 20
        if row['atende_fit_cultural'] == 1: 
            score += 30
        if row['atende_salario'] == 1: 
            score += calcular_score_salario(row, vagas_info, candidato_vaga_info)
        if row['soma_competencias_atendidas'] >= 1:  
            score += calcular_score_competencia(row, vagacompetencias_info)  

        # Adicionar score à lista
        scores.append({'vaga_id': row['vaga_id'], 'candidato_id': row['candidato_id'], 'score': score})

    # Criar DataFrame com os scores
    df_scores = pd.DataFrame(scores) 
    df_scores = df_scores.sort_values('vaga_id')

    return df_scores

In [13]:

# Executar a função principal
calcular_scores(df_min_candidatos)

/tmp/ipykernel_846/1390427362.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_incosistencias['score'] = 1


,vaga_id,candidato_id,score
565,3,963,4.29
390,3,620,12.86
224,3,358,8.57
426,3,693,4.29
528,4,886,20.00
...,...,...,...
24,99,43,5.00
516,99,872,5.00
45,99,77,20.00
522,100,877,93.81
